# 高级RAG技术汇总

检索增强生成（Retrieval-Augmented Generation，又称RAG）
- 通过检索LLMs之外的数据源来支持其生成答案。
- RAG=搜索+LLM提示，根据用户的查询要求，LLMs会使用搜索算法从外部数据源获取上下文信息
- 最后，查询和检索到的上下文合成后送入到LLM的提示中。

简言之就是通过问题找到回答问题的相关文本，从利用大模型和相关文本生成问题答案

## 一、Naive RAG
### 基础RAG流程和Prompt
1. 将原始数据文本分割成块
2. 将分割成块的文本向量化，并存入向量数据库中
3. 通过问题从向量数据库中检索相关文本
4. 将检索到的相关文本和问题一起作为提示输入到LLM中，生成答案

示例：
```
def question_answering(context, query):
    prompt = f"""
                Give the answer to the user query delimited by triple backticks ```{query}```\
                using the information given in context delimited by triple backticks ```{context}```.\
                If there is no relevant information in the provided context, try to answer yourself, 
                but tell user that you did not have any relevant context to base your answer on.
                Be concise and output the answer of size less than 80 tokens.
                """

    response = get_completion(instruction, prompt, model="gpt-3.5-turbo")
    answer = response.choices[0].message["content"]
    return answer
```

提高RAG pipeline最经济的方式就是Prompt工程，可以参考OpenAI提示工程指南。  
https://platform.openai.com/docs/guides/prompt-engineering/strategy-write-clear-instructions

## 二、Advanced RAG
### 2.1 Chunking & vectorization 分块与向量化
1. Chunking 分块
    - 文章粒度：语义信息丰富，具体，繁杂，包含大量无用信息，在相似度检索上不易于检索
    - 段落粒度：语义较为完整，较为易于检索
    - 句子粒度：易于检索，但是语义信息可能不完整
2. Vectorization 向量化
    - 选择Embedding模型
    - Embedding模型排行榜：https://huggingface.co/spaces/mteb/leaderboard
### 2.2 Search index 检索索引
1. Vector store index 向量存储索引
    - 最简单的实现是使用一个平面索引——暴力计算查询向量和所有块向量之间的距离。
    - 如果向量数量超过10000多个时，可以采用为高效检索而优化的向量索引，如faiss、nmslib或annoy，使用一些近似最近邻实现，如clustring、trees或HNSW算法。
    - 还有一些托管解决方案，如OpenSearch或ElasticSearch和vector数据库（比如Pinecone、Weaviate或Chroma）。
    - 根据索引选择、数据和搜索需要，还可以将元数据与向量一起存储，然后使用元数据过滤器可以搜索例如：某些日期或源中的信息。
2. Hierarchical indices 层次索引
    - 对于大型数据库，一种有效的方法是创建两个索引（一个由摘要组成，另一个由文档块组成），并分两步进行搜索，首先通过摘要过滤出相关文档，然后在该相关组中进行搜索。
3. Hypothetical Questions and HyDE 假设性问题与HyDE
    - 要求LLM为每个分块生成一个问题，并将这些问题嵌入向量中。在运行时对问题向量的索引执行查询搜索（将分块向量替换为索引中的问题向量），然后在检索后路由到原始文本区块并将其作为上下文发送给LLM以获得答案。这种方法提高了搜索质量，因为查询和假设问题之间的语义相似度比实际块更高。
    - 还有一种称为HyDE的反向逻辑方法—要求LLM生成给定查询的假设响应，然后使用其向量和查询向量来提高搜索质量。https://boston.lti.cs.cmu.edu/luyug/HyDE/HyDE.pdf
4. Context enrichment 丰富上下文
    - 检索更小的块以获得更好的搜索质量，但要将周围的上下文相加以供LLM推理
        - Sentence Window Retrieval 句子窗口检索
            - 在检索到的句子前后对上下文窗口进行k个句子的扩展，然后将扩展后的上下文发送给LLM。
        - Auto-merging Retriever (自动合并检索器)
            - 文档被分割成更小的子块，引用更大的父块。
5. Fusion retrieval or hybrid search 融合检索或混合搜索
    - 一个相对传统的想法是，可以从两个世界中取其精华-基于关键字的老式搜索-稀疏检索算法，如tf-idf或搜索行业标准BM25-和现代语义或向量搜索，并将其组合到一个检索结果。这里唯一的技巧是将检索到的结果与不同的相似性分数正确地结合起来——这个问题通常通过使用 Reciprocal Rank Fusion算法来解决，将检索到的结果重新排序以获得最终输出。https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - 混合或融合搜索通常通过两种互补的搜索算法相结合，同时考虑查询与存储文档之间的语义相似度和关键字匹配，从而提供更好的检索结果。
### 2.3 Reranking & filtering 重新排序与过滤
- 对检索到的结果进行重新排序，或者过滤，获得质量更高的检索文本
### 2.4 Query transformations 查询转换
- 如果查询很复杂，LLM可以将其分解为多个子查询
### 2.5 Chat Engine 聊天引擎
- 进行多轮对话聊天需要跟踪会话、回指或记录历史聊天记录。该方法采用查询压缩技术，结合聊天环境和用户查询，解决了该问题。
- 几种实现上下文压缩方法：
    - ContextChatEngine：首先检索与用户查询相关的上下文，然后将其与缓存中的聊天历史一起输入给LLM，以便LLM在生成下一个答案的同时考虑历史聊天记录
    - CondensePlusContextMode（）—在每个交互中，聊天历史和最后一条消息被压缩成一个新的查询，然后将这个查询建立索引再进行检索，检索到的上下文与原始用户消息一起传递给LLM以生成答案。
### 2.6 Query Routing 查询路由
- 查询路由是LLM支持的决策步骤，决定在给定用户查询的情况下接下来要做什么——通常是总结、对某些数据索引执行搜索或尝试多个不同的路由，然后在单个答案中综合它们的输出。
查询路由器可以选择一个索引或者数据存储来发送用户的查询。或者有多个数据源，例如，经典向量存储和图形数据库或关系数据库，或者您有一个索引层次结构—对于多文档存储来说，一个非常经典的例子是摘要索引和另一个文档块向量索引。定义查询路由器包括设置它可以做出的选择。通过LLM调用执行路由选项的选择，以预定义格式返回其结果，将查询路由到给定的索引，或者，如果我们采用不相关行为，则路由到子链或甚至其他代理，如下面的多文档代理方案所示。
### 2.7 Agents in RAG RAG中的代理
- 代理几乎自第一个LLM API发布以来就一直存在——其思想是提供一个LLM，能够推理，具有一组工具和要完成的任务
### 2.8 Response synthesiser 响应合成器
响应综合的主要方法有：
- 1、通过逐块向LLM发送检索到的上下文，迭代地细化答案；
- 2、总结检索到的上下文以适应提示；
- 3、根据不同的上下文块生成多个答案，并将其串联或汇总。
## 三、Encoder and LLM fine-tuning
需要注意的是：使用专业研究团队训练开源模型收集、清理和验证的大型数据集，并使用小型合成数据集进行快速微调，可能会降低模型的通用能力。  
## 四、Evaluation
评估指标
- 总体答案相关性
- 答案有根据性
- 可信度
- 缉拿索道的上下文相关性

1.Ragas使用可信度和答案相关性（faithfulness and answer relevance ）作为生成的答案质量指标，并使用经典上下文精确度P和召回率R （precision and recall）作为RAG方案的检索部分指标。  
2.在Andrew NG最近发布课程“构建和评估高级RAG”中，LlamaIndex和评估框架Truelens提出RAG三元组——检索到上下文的与查询的相关性、有根据性（所提供的上下文支持LLM答案的多少）和回答与查询相关性。（retrieved context relevance to the query, groundedness  and answer relevance to the query.）https://learn.deeplearning.ai/building-evaluating-advanced-rag/关键且最可控的度量是检索到的上下文相关性（retrieved context relevance）—基本上，上面描述的高级RAG管道的第1–7部分以及编码器和Ranker微调部分旨在改进此指标，而第8部分和LLM微调则侧重于答案相关性和基础性。  
3.非常简单的检索器评估Pipeline的例子，并将其应用于编码器微调部分。https://github.com/run-llama/finetune-embedding/blob/main/evaluate.ipynb  
4.OpenAI cookbook中展示了一种更高级的方法，该方法不仅考虑了命中率，而且还考虑了平均倒数秩（ Mean Reciprocal Rank）（一种常见的搜索引擎度量）以及生成的答案度量（如忠实度和相关性）（faithfulness abd relevance）。（https://github.com/openai/openai-cookbook/blob/main/examples/evaluation/Evaluate_RAG_with_LlamaIndex.ipynb）  
5.LangChain有一个非常高级的评估框架LangSmith（https://docs.smith.langchain.com/），其中可以实现定制的评估器，并监视RAG管道中的运行状况，以使系统更加透明。在LlamaIndex中，有一个rag_evaluator llama pack包（https://github.com/run-llama/llama-hub/tree/dac193254456df699b4c73dd98cdbab3d1dc89b0/llama_hub/llama_packs/rag_evaluator），它提供了一个快速工具，可以使用公共数据集评估管道。